In [2]:
import sys
sys.path.append('/home/cara/Documents/reddit_analysis_code')
import numpy as np
import pandas as pd
from reddit_dataclass import RedditData as reddit
import pickle
import matplotlib.pyplot as plt

In [3]:
sentiment_datasets = pickle.load(open('sentiment_5_datasets.p', 'rb'))

In [4]:
thread_cols = ['thread_id', 'id', 'timestamp','author','domain','parent',
       'score', 'subject_sentiment_score', 'body_sentiment_score', 'date']

In [18]:
"""
for key in sentiment_datasets:
    sentiment_datasets[key].data['parent'] = sentiment_datasets[key].data['parent'].str[3:]
pickle.dump(sentiment_datasets, open('sentiment_datasets.p', 'wb'))
"""

In [ ]:
thread_datasets = {}

for key in sentiment_datasets:
    print(f'### WORKING ON {key} ###')

    sentiment_data = sentiment_datasets[key].data
    thread_data = sentiment_data[thread_cols]
    thread_data['level'] = -1

    parent_post_id = sentiment_data[sentiment_data.thread_id == sentiment_data.id].id

    thread_data.loc[thread_data.id.isin(parent_post_id), 'level'] = 0

    children = {}

    children[0] = sentiment_data[sentiment_data.parent == sentiment_data.thread_id]
    children[0]['parent_comment'] = children[0].id
    children[0]['level'] = 1

    i = 0
    while len(children[i]) > 0:

        print(f"level {i}  number of children: {len(children[i])}")
        i += 1
        children[i] = sentiment_data[sentiment_data.parent.isin(children[i-1].id)]
        children[i]['level'] = i + 1
        parent_mapper = dict(zip(
            children[i-1].id, children[i-1].parent_comment
        ))
        children[i]["parent_comment"] = children[i].parent.map(parent_mapper)
    
    thread_data['parent_comment'] = np.nan

    for lvl in children:
        thread_data.loc[
            thread_data.id.isin(children[lvl].id), 
            'level'] = lvl + 1
        if lvl == 0:
            thread_data.loc[
                thread_data.id.isin(children[lvl].id),
                'parent_comment'] = thread_data[
                    thread_data.id.isin(children[lvl].id)].id
        else:
            parent_mapper = dict(zip(
                children[lvl].id, children[lvl].parent_comment
            ))
            thread_data.loc[
                thread_data.id.isin(children[lvl].id),
                'parent_comment'] = thread_data[
                    thread_data.id.isin(children[lvl].id)].id.map(
                        parent_mapper)
    
    thread_datasets[key] = thread_data
    


In [6]:
pickle.dump(thread_datasets, open('thread_5_data.p', 'wb'))

In [7]:
unclassified = {}
for key in thread_datasets:
    unclassified[key] = thread_datasets[key][thread_datasets[key].level == -1]

In [16]:
print('subreddit  sentiment  thread_entries     unclassified')
for key in thread_datasets:
    print(f"{key} {len(sentiment_datasets[key].data)}  {len(thread_datasets[key])} {len(unclassified[key])}")

subreddit  sentiment  thread_entries     unclassified
books 105291  105291 13169
conspiracy 423958  423958 46573
crypto 444397  444397 36908
thedonald 1543  1543 233
politics 6428330  6428330 2788540


In [ ]:
# check if spread over a range of dates
for key in unclassified:
    date_str = ""
    for date in unclassified[key].date.unique():
        date_str += f"{date}    "
    print(f"## {key} ##")
    print(date_str)

In [ ]:
removed_parent_activity = {}
for key in sentiment_datasets:
    removed_parent_activity[key] = unclassified[key][unclassified[key].parent.isin(sentiment_datasets[key].removed.id)]
    print(f"{key}   {len(removed_parent_activity[key])}")

In [11]:
# save thread data and unclassified data
clean_thread_data = {}
for key in thread_datasets:
    clean_thread_data[key] = thread_datasets[key][thread_datasets[key].level != -1]

In [12]:
pickle.dump(clean_thread_data, open('clean_5_thread_data.p', 'wb'))
pickle.dump(unclassified, open('unclassified_5_thread_data.p', 'wb'))

In [59]:
clean_thread_data = pickle.load(open('clean_thread_data.p', 'rb'))
unclassified = pickle.load(open('unclassified_thread_data.p', 'rb'))

In [17]:
# BUT also need to remove threads that have top comment but no post
for key in clean_thread_data:
    to_remove = clean_thread_data[key][~
            clean_thread_data[key].thread_id.isin(
                clean_thread_data[key].id)]
    unclassified[key] = pd.concat(
        (unclassified[key],
        to_remove))
    clean_thread_data[key] = clean_thread_data[key][
        clean_thread_data[key].thread_id.isin(clean_thread_data[key].id)
        ]
    print(key)
    print(f"to remove: {len(to_remove)}")
    print(f"clean: {len(clean_thread_data[key])}")

books
to remove: 0
clean: 92122
conspiracy
to remove: 0
clean: 377385
crypto
to remove: 0
clean: 407489
thedonald
to remove: 0
clean: 1310
politics
to remove: 0
clean: 3639790


In [ ]:
unclassified['thedonald']

In [18]:
pickle.dump(clean_thread_data, open('clean_5_thread_data.p', 'wb'))
pickle.dump(unclassified, open('unclassified_5_thread_data.p', 'wb'))

: 